In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np

def create_harmonic_oscillator_circuit(phi0, phi1, phi2, error_qubit=None):
    # Create registers
    data_q = QuantumRegister(2, 'data')          # Original data qubits
    ancilla_q = QuantumRegister(4, 'ancilla')    # Ancillas for encoding (2 per logical qubit)
    syndrome_q = QuantumRegister(2, 'syndrome')  # Syndrome measurement qubits
    cr = ClassicalRegister(6, 'cr')              # Classical register (4 syndrome + 2 data)
    
    qc = QuantumCircuit(data_q, ancilla_q, syndrome_q, cr)
    
    # Initialize data_q in (|00> + |01> + |10>)/sqrt(3)
    psi = [1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0]  # [|00>, |01>, |10>, |11>]
    qc.initialize(psi, data_q)
    
    # Apply phases using the masking technique
    qc.x(data_q[0])
    qc.x(data_q[1])
    qc.cp(phi0, data_q[0], data_q[1])
    qc.x(data_q[1])
    qc.x(data_q[0])
    
    qc.x(data_q[0])
    qc.cp(phi1, data_q[0], data_q[1])
    qc.x(data_q[0])
    
    qc.x(data_q[1])
    qc.cp(phi2, data_q[0], data_q[1])
    qc.x(data_q[1])
    
    # =============================================
    # 2. Encode both qubits using phase-flip code
    # =============================================
    # Encode first qubit (data_q[0] -> data_q[0], ancilla_q[0], ancilla_q[1])
    qc.h(data_q[0])
    qc.cx(data_q[0], ancilla_q[0])
    qc.cx(data_q[0], ancilla_q[1])
    qc.h([data_q[0], ancilla_q[0], ancilla_q[1]])
    
    # Encode second qubit (data_q[1] -> data_q[1], ancilla_q[2], ancilla_q[3])
    qc.h(data_q[1])
    qc.cx(data_q[1], ancilla_q[2])
    qc.cx(data_q[1], ancilla_q[3])
    qc.h([data_q[1], ancilla_q[2], ancilla_q[3]])
    
    # =============================================
    # 3. (Optional) Inject error for testing
    # =============================================
    if error_qubit is not None:
        qc.z(error_qubit)  # Introduce phase error
    
    # =============================================
    # 4. Measure stabilizers for error detection
    # =============================================
    # First logical qubit stabilizers
    qc.h(syndrome_q[0])
    qc.cx(syndrome_q[0], data_q[0])
    qc.cx(syndrome_q[0], ancilla_q[0])
    qc.h(syndrome_q[0])
    qc.measure(syndrome_q[0], cr[0])
    
    qc.h(syndrome_q[1])
    qc.cx(syndrome_q[1], data_q[0])
    qc.cx(syndrome_q[1], ancilla_q[1])
    qc.h(syndrome_q[1])
    qc.measure(syndrome_q[1], cr[1])
    
    # Reset syndrome qubits for reuse
    qc.reset(syndrome_q)
    
    # Second logical qubit stabilizers
    qc.h(syndrome_q[0])
    qc.cx(syndrome_q[0], data_q[1])
    qc.cx(syndrome_q[0], ancilla_q[2])
    qc.h(syndrome_q[0])
    qc.measure(syndrome_q[0], cr[2])
    
    qc.h(syndrome_q[1])
    qc.cx(syndrome_q[1], data_q[1])
    qc.cx(syndrome_q[1], ancilla_q[3])
    qc.h(syndrome_q[1])
    qc.measure(syndrome_q[1], cr[3])
    
    # =============================================
    # 5. Apply error correction (using if_test)
    # =============================================
    # First logical qubit correction
    with qc.if_test((cr[0], 1)):
        with qc.if_test((cr[1], 1)):
            qc.x(data_q[0])  # Apply X correction for phase error
        with qc.if_test((cr[1], 0)):
            qc.x(ancilla_q[0])
    with qc.if_test((cr[0], 0)):
        with qc.if_test((cr[1], 1)):
            qc.x(ancilla_q[1])

    # Second logical qubit correction
    with qc.if_test((cr[2], 1)):
        with qc.if_test((cr[3], 1)):
            qc.x(data_q[1])  # Apply X correction for phase error
        with qc.if_test((cr[3], 0)):
            qc.x(ancilla_q[2])
    with qc.if_test((cr[2], 0)):
        with qc.if_test((cr[3], 1)):
            qc.x(ancilla_q[3])
    
    # =============================================
    # 6. Decode the logical qubits
    # =============================================
    # First logical qubit
    qc.h([data_q[0], ancilla_q[0], ancilla_q[1]])
    qc.cx(data_q[0], ancilla_q[0])
    qc.cx(data_q[0], ancilla_q[1])
    
    # Second logical qubit
    qc.h([data_q[1], ancilla_q[2], ancilla_q[3]])
    qc.cx(data_q[1], ancilla_q[2])
    qc.cx(data_q[1], ancilla_q[3])
    
    # =============================================
    # 7. Final measurement
    # =============================================
    qc.measure(data_q[0], cr[4])
    qc.measure(data_q[1], cr[5])
    
    return qc

def simulate_and_analyze(qc, shots=1000):
    # Simulate the circuit
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    
    # Analyze results
    data_counts = {}
    syndrome_stats = {}
    
    for bitstring, count in counts.items():
        # Extract components from bitstring (order: syndrome[3], syndrome[2], syndrome[1], syndrome[0], data[1], data[0])
        # Qiskit uses little-endian: first bit is last measured
        syndrome = bitstring[:4]  # First 4 characters
        data = bitstring[4:6]     # Next 2 characters
        
        # Count data results
        data_counts[data] = data_counts.get(data, 0) + count
        
        # Count syndrome patterns
        syndrome_stats[syndrome] = syndrome_stats.get(syndrome, 0) + count
    
    # Calculate percentages
    total = sum(data_counts.values())
    print("\nFinal data qubit measurements:")
    for state in ['00', '01', '10', '11']:
        count = data_counts.get(state, 0)
        print(f"|{state}⟩: {count} times ({count/total*100:.1f}%)")
    
    print("\nSyndrome patterns observed:")
    for syndrome, count in syndrome_stats.items():
        print(f"{syndrome}: {count} times")
    
    # Calculate error rate
    error_rate = sum(count for syn, count in syndrome_stats.items() if syn != "0000") / total * 100
    print(f"\nEstimated error rate: {error_rate:.1f}%")
    
    return data_counts, syndrome_stats

# Example usage
if __name__ == "__main__":
    # Phase parameters
    phi0, phi1, phi2 = 0.5, 1.0, 1.5
    
    print("=====================")
    print("Noiseless Simulation")
    print("=====================")
    qc_noiseless = create_harmonic_oscillator_circuit(phi0, phi1, phi2)
    simulate_and_analyze(qc_noiseless)
    
    print("\n\n=======================")
    print("With Injected Error")
    print("=======================")
    # Inject error on physical qubit 3 (ancilla for second logical qubit)
    qc_with_error = create_harmonic_oscillator_circuit(phi0, phi1, phi2, error_qubit=3)
    simulate_and_analyze(qc_with_error)




Noiseless Simulation

Final data qubit measurements:
|00⟩: 1000 times (100.0%)
|01⟩: 0 times (0.0%)
|10⟩: 0 times (0.0%)
|11⟩: 0 times (0.0%)

Syndrome patterns observed:
0100: 547 times
1100: 252 times
1000: 118 times
0000: 83 times

Estimated error rate: 91.7%


With Injected Error

Final data qubit measurements:
|00⟩: 0 times (0.0%)
|01⟩: 0 times (0.0%)
|10⟩: 1000 times (100.0%)
|11⟩: 0 times (0.0%)

Syndrome patterns observed:
0100: 506 times
1000: 151 times
0000: 100 times
1100: 243 times

Estimated error rate: 90.0%
